### MelanomaDataset

In [ ]:
class MelanomaDataset(Dataset):
    '''Melanoma Dataset'''

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.image_paths = self.df.filepath.values

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image'].astype(np.float32)
        else:
            image = image.astype(np.float32)

        image = image.transpose(2, 0, 1)

        label = torch.tensor(self.df.target.values[index])

        return image, label

### AttConv

In [ ]:
class AttConv(nn.Module):
    def __init__(self, num_in_chan=1, num_out_chan=6, kernel_size=5, stride=1):
        super(AttConv, self).__init__()
        self.kernel_size=kernel_size
        self.num_in_chan=num_in_chan
        self.num_out_chan=num_out_chan
        self.stride=stride
        self.BU_weights = nn.Parameter(torch.HalfTensor(1,num_in_chan*kernel_size**2, 1, num_out_chan))
        init.kaiming_uniform_(self.BU_weights, a=np.sqrt(5))
        self.TD_weights = nn.Parameter(self.BU_weights.data.detach().clone())
        
        self.BU_bias = nn.Parameter(torch.randn(1,1,1,num_out_chan)*0.1)
        self.TD_bias = nn.Parameter(torch.randn(1,num_in_chan,1,1)*0.1)
        
    def normalize_att_weights(self, in_spat_dim):
        batch_size = self.att_weights.shape[0]        
        num_wins = self.att_weights.shape[-1]
        aw_sum = F.unfold(F.fold(self.att_weights, in_spat_dim, self.kernel_size,stride=self.stride), self.kernel_size, stride=self.stride) #Fold into image domain (which automatically computes the sum per pixel), and then unfold again into conv windows    
        self.att_weights = self.att_weights/aw_sum #Normalize weights by their sum over possible parents
        self.att_weights = self.att_weights.view(batch_size, 1, self.kernel_size**2, num_wins).expand(batch_size, self.num_in_chan, self.kernel_size**2, num_wins).reshape(batch_size, self.num_in_chan*self.kernel_size**2, num_wins)
        
        
    def forward(self, x, num_iter=4):
        batch_size = x.shape[0]
        device = x.device
        in_spat_dim = list(x.shape[-2:])                
        assert in_spat_dim[0]==in_spat_dim[1], 'Only square images are supported'
        x_wins = F.unfold(x.view(batch_size,self.num_in_chan,*in_spat_dim), self.kernel_size, stride=self.stride)
        x_wins = x_wins.type(torch.half)
        out_spat_dim = np.int(np.sqrt(x_wins.shape[-1]))
        self.att_weights = torch.ones([batch_size, self.kernel_size**2, x_wins.shape[-1]], device=device, dtype=torch.half)        
        self.normalize_att_weights(in_spat_dim)
        
        
        for i in range(num_iter):
            y = F.relu((x_wins.unsqueeze(-1)*self.att_weights.unsqueeze(-1)*self.BU_weights).sum(1,True) + self.BU_bias)
            pred = (y*self.TD_weights).sum(-1).view(batch_size,self.num_in_chan,self.kernel_size**2, -1) + self.TD_bias            
            self.att_weights = ((pred*x_wins.view(batch_size,self.num_in_chan,self.kernel_size**2, -1)).sum(1) / np.sqrt(self.num_in_chan)).exp()
            self.normalize_att_weights(in_spat_dim)   

        y = y.view(batch_size, out_spat_dim, out_spat_dim, self.num_out_chan).permute(0,3,1,2)
        
        return y

    
        

In [ ]:
plt.style.use(['classic'])
np.random.seed(13)
torch.manual_seed(13)

# Attention

In [ ]:
image_size = 128

train_transforms = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=5),
            albumentations.MedianBlur(blur_limit=5),
            albumentations.GaussianBlur(blur_limit=5),
            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

        albumentations.OneOf([
            albumentations.OpticalDistortion(distort_limit=1.0),
            albumentations.GridDistortion(num_steps=5, distort_limit=1.),
            albumentations.ElasticTransform(alpha=3),
        ], p=0.7),

        albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])

test_transforms = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1701: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1727: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1852: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:645: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


In [ ]:
train_set = MelanomaDataset(train_data, transform=train_transforms)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True,  num_workers=2)

valid_set = MelanomaDataset(valid_data, transform=test_transforms)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=False, num_workers=1)

In [ ]:
epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# loss = torch.nn.CrossEntropyLoss(weight=class_weight_ts)
loss = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

loss = loss.to(device)

In [ ]:
running_loss = np.zeros((epochs, len(train_loader)))
running_train_acc = np.zeros(epochs)
running_valid_acc = np.zeros(epochs)
# auc = np.zeros(epochs)

best_accuracy = 0

#### Attention with upsampling

In [ ]:
image_size = 128

train_transforms = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=5),
            albumentations.MedianBlur(blur_limit=5),
            albumentations.GaussianBlur(blur_limit=5),
            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

        albumentations.OneOf([
            albumentations.OpticalDistortion(distort_limit=1.0),
            albumentations.GridDistortion(num_steps=5, distort_limit=1.),
            albumentations.ElasticTransform(alpha=3),
        ], p=0.7),

        albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])

test_transforms = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1701: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1727: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1852: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:645: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


In [ ]:
# train_set = MelanomaDataset(train_data, transform=train_transforms)
# train_loader = DataLoader(train_set, batch_size=32, shuffle=True,  num_workers=2)

# valid_set = MelanomaDataset(valid_data, transform=test_transforms)
# valid_loader = DataLoader(valid_set, batch_size=16, shuffle=False, num_workers=1)
# --

train_set = MelanomaDataset(train_data, transform=train_transforms)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True,  num_workers=2)

valid_set = MelanomaDataset(valid_data, transform=test_transforms)
valid_loader = DataLoader(valid_set, batch_size=16, shuffle=False, num_workers=1)

In [ ]:
class AttAllConvNet(nn.Module):
    def __init__(self, input_size, n_classes=10, **kwargs):
        super(AttAllConvNet, self).__init__()
        # self.conv1 = nn.Conv2d(input_size, 96, 3, padding=1)
        self.conv1 = AttConv(num_in_chan=3, num_out_chan=96, kernel_size=3)
        self.conv2 = nn.Conv2d(96, 96, 3, padding=1)
        self.conv3 = nn.Conv2d(96, 96, 3, padding=1, stride=2)
        self.conv4 = nn.Conv2d(96, 192, 3, padding=1)
        self.conv5 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv6 = nn.Conv2d(192, 192, 3, padding=1, stride=2)
        self.conv7 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv8 = nn.Conv2d(192, 192, 1)

        self.class_conv = nn.Conv2d(192, n_classes, 1)


    def forward(self, x):
        x_drop = F.dropout(x, .2)
        conv1_out = self.conv1(x_drop)
        conv2_out = F.relu(self.conv2(conv1_out))
        conv3_out = F.relu(self.conv3(conv2_out))
        conv3_out_drop = F.dropout(conv3_out, .5)
        conv4_out = F.relu(self.conv4(conv3_out_drop))
        conv5_out = F.relu(self.conv5(conv4_out))
        conv6_out = F.relu(self.conv6(conv5_out))
        conv6_out_drop = F.dropout(conv6_out, .5)
        conv7_out = F.relu(self.conv7(conv6_out_drop))
        conv8_out = F.relu(self.conv8(conv7_out))

        class_out = F.relu(self.class_conv(conv8_out))
        pool_out = F.adaptive_avg_pool2d(class_out, 1)
        pool_out.squeeze_(-1)
        pool_out.squeeze_(-1)
        return pool_out


att_allconv = AttAllConvNet(input_size=3, n_classes=2)
att_allconv.eval()

AttAllConvNet(
  (conv1): AttConv()
  (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (class_conv): Conv2d(192, 2, kernel_size=(1, 1), stride=(1, 1))
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from torchsummary import summary
att_allconv.to(device)
summary(att_allconv, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           AttConv-1         [-1, 96, 126, 126]               0
            Conv2d-2         [-1, 96, 126, 126]          83,040
            Conv2d-3           [-1, 96, 63, 63]          83,040
            Conv2d-4          [-1, 192, 63, 63]         166,080
            Conv2d-5          [-1, 192, 63, 63]         331,968
            Conv2d-6          [-1, 192, 32, 32]         331,968
            Conv2d-7          [-1, 192, 32, 32]         331,968
            Conv2d-8          [-1, 192, 32, 32]          37,056
            Conv2d-9            [-1, 2, 32, 32]             386
Total params: 1,365,506
Trainable params: 1,365,506
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 42.31
Params size (MB): 5.21
Estimated Total Size (MB): 47.70
-------------------------------------

In [ ]:
pd.DataFrame(list(running_loss.transpose()), columns=['Epoch 1']).to_csv(
    os.path.join(model_dir, 'running_loss_attallconv.csv'), index=False)
pd.DataFrame(running_valid_acc).to_csv(
    os.path.join(model_dir, 'val_acc_resnet_attallconv.csv'), index=False)
pd.DataFrame(running_train_acc).to_csv(
    os.path.join(model_dir, 'train_acc_resnet_attallconv.csv'), index=False)

result = pd.read_csv('/content/drive/MyDrive/thesis/models/test_attallconv_upsampled.csv')
y_true = result['True Label'].values
y_pred = result['Predicted Label'].values

cfm = confusion_matrix(y_true, y_pred)
print(cfm)
plt.title('Melanoma detection with 128 image data')
sns.heatmap(cfm, annot=True, cmap='Blues', xticklabels=['pred_Benign', 'pred_Malignant'], yticklabels=['Benign','Malignant'])
plt.show()

In [ ]:
att_model = models.alexnet(pretrained=False)
att_model.features = nn.Sequential(
    # nn.BatchNorm2d(3),
    AttConv(3, 32, 5),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(32, 96, kernel_size=5, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(96, 192, kernel_size=3, padding=1), nn.ReLU(inplace=True), 
    nn.Conv2d(192, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    )

att_model.classifier = nn.Sequential(
    nn.Dropout(),
    nn.Linear(in_features=128*6*6, out_features=128*4*4, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128*4*4, out_features=128*4*4, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=128*4*4, out_features=2, bias=True),
  )

att_model.eval()


AlexNet(
  (features): Sequential(
    (0): AttConv()
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4608, out_features=2048, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_fea

In [ ]:
summary(att_model, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           AttConv-1         [-1, 32, 124, 124]               0
         MaxPool2d-2           [-1, 32, 61, 61]               0
            Conv2d-3           [-1, 96, 59, 59]          76,896
              ReLU-4           [-1, 96, 59, 59]               0
         MaxPool2d-5           [-1, 96, 29, 29]               0
            Conv2d-6          [-1, 192, 29, 29]         166,080
              ReLU-7          [-1, 192, 29, 29]               0
            Conv2d-8          [-1, 128, 29, 29]         221,312
              ReLU-9          [-1, 128, 29, 29]               0
           Conv2d-10          [-1, 128, 29, 29]         147,584
             ReLU-11          [-1, 128, 29, 29]               0
        MaxPool2d-12          [-1, 128, 14, 14]               0
AdaptiveAvgPool2d-13            [-1, 128, 6, 6]               0
          Dropout-14                 [-

In [ ]:
# sys.stdout = open(os.path.join(model_dir, 'AttConv_output(1).txt'), 'w')

start_time = time.time()

for epoch in range(epochs):
    train_correct = 0
    valid_correct = 0
    scheduler.step()

    for i, (images, labels) in enumerate(tqdm(train_loader)):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward
        output = att_model(images).to(device)
        # Calculate loss
        err = loss(output, labels)
        # Backward
        err.backward()
        # Update weights
        optimizer.step()

        _, train_predict = torch.max(output.data, 1)
        train_correct += (train_predict.cpu().numpy() == labels.cpu().numpy()).sum()

        # Show training loss at every 200 data points
        if (i+1) % 200 == 0:
            print( "[Epoch {} / {}] At step {} of {}, Training loss = {:.4f}".
                format(epoch + 1, epochs, i + 1, len(train_loader), err.item()) )
        
        # Add to running loss
        running_loss[epoch, i] = err.item()

    # Accuracy at each epoch: add to tracking
    running_train_acc[epoch] = train_correct / len(train_set) * 100

    att_model.eval()


    for i, (images, labels) in enumerate(tqdm(valid_loader)):
        images, labels = images.to(device), labels.to(device)
        output = att_model(images).to(device)

        _, valid_predict = torch.max(output.data, 1)
        valid_correct += (valid_predict.cpu().numpy() == labels.cpu().numpy()).sum()

        if epoch == (epochs - 1):
            for i in range(len(labels)):
                pred_label.append(valid_predict.cpu().numpy()[i])
                actual_label.append(labels.cpu().numpy()[i])
    
    # Accuracy at each epoch: add to tracking, print
    running_valid_acc[epoch] = valid_correct / len(valid_set) * 100

    print( "Accuracy of network on test set at epoch {} of {}: {}/{} = {:.2f}%".
          format(epoch + 1, epochs, valid_correct, len(valid_set), (valid_correct / len(valid_set) * 100)) )

    if epoch == (epochs - 1):
        pd.DataFrame(list(zip(actual_label, pred_label)), columns=['True Label', 'Predicted Label']).to_csv(os.path.join(model_dir, '210525valid_result(att_org_trans).csv'), index=False)

    print("=================================")

end_time = time.time()
print("training time: {:0.1f} sec.".format(end_time - start_time))
